# Mamdani Fuzzy Inference System

**Classic Tipping Problem**

In [ ]:
!pip install pyfuzzy-toolbox -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fuzzy_systems.core import LinguisticVariable
from fuzzy_systems.inference import MamdaniSystem

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

### 1. Define Input Variables

In [ ]:
# Service quality: poor, good, excellent
service = LinguisticVariable('service', universe=(0, 10))
service.add_term('poor', 'triangular', (0, 0, 5))
service.add_term('good', 'triangular', (0, 5, 10))
service.add_term('excellent', 'triangular', (5, 10, 10))

# Food quality: bad, good, delicious
food = LinguisticVariable('food', universe=(0, 10))
food.add_term('bad', 'triangular', (0, 0, 5))
food.add_term('good', 'triangular', (0, 5, 10))
food.add_term('delicious', 'triangular', (5, 10, 10))

# Visualize inputs
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
service.plot(ax=ax1, show=False)
ax1.set_title('Service Quality')
food.plot(ax=ax2, show=False)
ax2.set_title('Food Quality')
plt.tight_layout()
plt.show()

### 2. Define Output Variable

In [ ]:
# Tip percentage: low, medium, high
tip = LinguisticVariable('tip', universe=(0, 30))
tip.add_term('low', 'triangular', (0, 0, 15))
tip.add_term('medium', 'triangular', (0, 15, 30))
tip.add_term('high', 'triangular', (15, 30, 30))

tip.plot(figsize=(12, 4), show=True)

### 3. Define Fuzzy Rules

In [ ]:
# Create Mamdani system
fis = MamdaniSystem()
fis.add_input_variable(service)
fis.add_input_variable(food)
fis.add_output_variable(tip)

# Add fuzzy rules
fis.add_rule("IF service IS poor OR food IS bad THEN tip IS low")
fis.add_rule("IF service IS good THEN tip IS medium")
fis.add_rule("IF service IS excellent OR food IS delicious THEN tip IS high")

print(f"Fuzzy Inference System created with {len(fis.rules)} rules")

### 4. Single Inference Example

In [ ]:
# Test with specific values
test_input = {'service': 7.5, 'food': 8.0}
result = fis.infer(test_input, method='centroid')

print(f"Input: service={test_input['service']}, food={test_input['food']}")
print(f"Output: tip={result['tip']:.2f}%")

# Visualize inference process
fis.plot_inference(test_input, figsize=(14, 10))

### 5. Surface Plot: Response Over Input Space

In [ ]:
# Generate surface plot
service_range = np.linspace(0, 10, 30)
food_range = np.linspace(0, 10, 30)
tip_surface = np.zeros((len(food_range), len(service_range)))

for i, f in enumerate(food_range):
    for j, s in enumerate(service_range):
        result = fis.infer({'service': s, 'food': f}, method='centroid')
        tip_surface[i, j] = result['tip']

# 3D surface plot
from mpl_toolkits.mplot3d import Axes3D
S, F = np.meshgrid(service_range, food_range)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(S, F, tip_surface, cmap='viridis', alpha=0.8)
ax.set_xlabel('Service Quality')
ax.set_ylabel('Food Quality')
ax.set_zlabel('Tip (%)')
ax.set_title('Mamdani FIS: Tipping Response Surface')
plt.colorbar(surf, shrink=0.5)
plt.show()

### 6. Contour Plot

In [ ]:
# 2D contour plot
plt.figure(figsize=(10, 8))
contour = plt.contourf(S, F, tip_surface, levels=15, cmap='viridis')
plt.colorbar(contour, label='Tip (%)')
plt.xlabel('Service Quality')
plt.ylabel('Food Quality')
plt.title('Tipping Decision Contour Map')
plt.grid(True, alpha=0.3)
plt.show()

### 🎯 Exercise: Test Different Scenarios

In [ ]:
# Test multiple scenarios
scenarios = [
    {'service': 2, 'food': 3, 'desc': 'Poor service, bad food'},
    {'service': 5, 'food': 5, 'desc': 'Average everything'},
    {'service': 8, 'food': 9, 'desc': 'Great experience'},
    {'service': 3, 'food': 9, 'desc': 'Poor service, great food'},
    {'service': 9, 'food': 3, 'desc': 'Great service, bad food'}
]

for scenario in scenarios:
    inputs = {'service': scenario['service'], 'food': scenario['food']}
    result = fis.infer(inputs, method='centroid')
    print(f"{scenario['desc']:30s} → Tip: {result['tip']:5.2f}%")